In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys 

In [3]:
sys.path.append(os.path.abspath('../src/'))

In [4]:
from postgis_raster_bridge.database import query_db

In [17]:
out_raster_query = """
 WITH 
                    
        
        q AS (
            
            SELECT ST_GeomFromText('POLYGON((16.821519 41.05911, 16.821519 41.152986, 16.903489 41.152986, 16.903489 41.05911, 16.821519 41.05911))', 4326) AS geom
        
        )
    ,  
        qp AS (
            SELECT 
                ST_Transform(q.geom, 32632) as geom
            FROM q
        )
    ,
                    
        q_ras AS (
            SELECT
                ST_Union(
                    ST_AsRaster(
                        q.geom,
                        ( 
        SELECT ST_SetBandNoDataValue(ST_MakeEmptyRaster(382, 419, 16.821519, 41.152986, 0.00019609619029165923, 0.0002461398756614878, 0, 0, 4326), 254)
    ),
                        '8BUI'::text, 
                        0, 
                        254
                    ),
                    'Max'
                ) AS ras
            FROM q
        )
    ,
                    
        q_poly AS (
            SELECT  
                (pp).geom AS geom
            FROM 
                (
                    SELECT ST_PixelAsPolygons(
                        q_ras.ras, 
                        1
                    ) pp
                    FROM q_ras
                ) a
        )
    ,
                    features AS (
        SELECT 
            ic.pro_com as pro_com,
            ic.comune as comune,
            ST_Area(ST_Intersection(ic.wkb_geometry, qp.geom)) as area,
            ST_Area(ic.wkb_geometry) as total_area,
            ST_Transform(ic.wkb_geometry, 4326) as geom,
            1 AS code
        FROM                 
            public.ist_comuni ic,                
            qp
        WHERE                 
            ST_Intersects(qp.geom, ic.wkb_geometry)
    ), 
	d_feat AS (
                            SELECT 
								ST_Union(f.geom) as geom
                            FROM
                                features f
                            WHERE 
                                f.code=1
							),
							
                         d_ras AS (
                            SELECT  
                                ST_Union(
                                    ST_AsRaster(
                                        df.geom,
                                        ( 
        SELECT ST_SetBandNoDataValue(ST_MakeEmptyRaster(382, 419, 16.821519, 41.152986, 0.00019609619029165923, 0.0002461398756614878, 0, 0, 4326), 254)
    ),
                                        '8BUI'::text, 
                                        1, 
                                        0,
                                        true
                                    ),
                                    'Max'
                                ) AS ras
                            FROM
                                d_feat df
                        )
						
-- 						,
                    
--                     data_point_1 AS (
                        
    
--                         SELECT 
--                             ST_Collect(ST_Centroid((pp).geom)) as geom
--                         FROM
--                             (
--                                 SELECT 
--                                     ST_PixelAsPoints(d_ras.ras, 1) pp
--                                 FROM d_ras
--                             ) a
--                         WHERE
--                             (pp).val=1
--                     )
                
               
-- 		SELECT COUNT(t.d) 
-- 			FROM ( 
-- 		SELECT ST_Dump(geom) as d from data_point_1 ) t				
				

, out_raster AS ( 
    SELECT  
                                ST_Union(
                                    ST_AsRaster(
                                        df.geom,
                                        ( 
        SELECT ST_SetBandNoDataValue(ST_MakeEmptyRaster(382, 419, 16.821519, 41.152986, 0.00019609619029165923, 0.0002461398756614878, 0, 0, 4326), 254)
    ),
                                        '8BUI'::text, 
                                        1, 
                                        0,
                                        true
                                    ),
                                    'Max'
                                ) AS ras
                            FROM
                                d_feat df
)

                SELECT 
                    ST_AsTIFF(
                        out_raster.ras,
                        'LZW'
                    ) as outraster
                FROM out_raster
"""

In [18]:
output_raster_content = output_raster_content = query_db(out_raster_query, cursor_factory=None)[0][0]

In [20]:
output_raster_filepath = str(os.path.abspath(r"test/test.tiff"))

In [21]:
# FLush Raster to Disk
with open(output_raster_filepath, 'wb') as f:
    f.write(bytes(output_raster_content))